In [16]:
import random

In [57]:
import numpy as np
import string
from scipy.special import logsumexp
import random

# Example candidate answers with confidence ratings for each clue
candidates = {
    '1A': ['FOUR', 'FIVE', 'FISH'],
    '2A': ['ORAL', 'OREO', 'ONCE'],
    '3A': ['USED', 'USER', 'UNIT'],
    '4A': ['RANK', 'RAMP', 'RAIN'],
    '1D': ['FURY', 'FIRE', 'FIRM'],
    '2D': ['OVAL', 'OMNI', 'ONCE'],
    '3D': ['RARE', 'RAMP', 'RANK'],
    '4D': ['LUSH', 'LEND', 'LION']
}

# Example normalized confidence ratings for each candidate
confidence_ratings = {
    '1A': {'FOUR': 0.95, 'FIVE': 0.05, 'FISH': 0.00},
    '2A': {'ORAL': 0.5, 'OREO': 0.3, 'ONCE': 0.2},
    '3A': {'USED': 0.6, 'USER': 0.25, 'UNIT': 0.15},
    '4A': {'RANK': 0.01, 'RAMP': 0.05, 'RAIN': 0.94},
    '1D': {'FURY': 0.7, 'FIRE': 0.2, 'FIRM': 0.1},
    '2D': {'OVAL': 0.4, 'OMNI': 0.35, 'ONCE': 0.25},
    '3D': {'RARE': 0.55, 'RAMP': 0.3, 'RANK': 0.15},
    '4D': {'LUSH': 0.65, 'LEND': 0.25, 'LION': 0.1}
}

class VariableNode:
    def __init__(self, position):
        self.position = position
        self.letters = list(string.ascii_uppercase)
        self.log_probs = np.log(np.full(len(self.letters), 1.0 / len(self.letters)))
        self.neighbors = []

    def add_neighbor(self, neighbor):
        self.neighbors.append(neighbor)

    def receive_message(self, factor_node, message):
        self.log_probs += message

    def normalize(self):
        self.log_probs -= logsumexp(self.log_probs)

class FactorNode:
    def __init__(self, clue, length, candidates, confidence_ratings):
        self.clue = clue
        self.length = length
        self.candidates = candidates
        self.confidence_ratings = confidence_ratings
        self.neighbors = []
        self.messages = {}

    def add_neighbor(self, variable_node):
        self.neighbors.append(variable_node)
        self.messages[variable_node.position] = np.zeros(len(variable_node.letters))

    def send_message(self, variable_node):
        idx = self.neighbors.index(variable_node)
        other_vars = self.neighbors[:idx] + self.neighbors[idx + 1:]
        message = np.zeros(len(variable_node.letters))
        for candidate in self.candidates:
            candidate_indices = [string.ascii_uppercase.index(char) for char in candidate]
            candidate_log_prob = np.log(self.confidence_ratings[candidate])

            for other_var in other_vars:
                other_idx = self.neighbors.index(other_var)
                candidate_log_prob += other_var.log_probs[candidate_indices[other_idx]]
               
            
            TEMP_BI = random.uniform(0, 0.3)
            
            # Incorporate bigram probabilities
            if idx > 0:  # Previous letter bigram
                prev_var = self.neighbors[idx - 1]
                prev_letter_idx = candidate_indices[idx - 1]
                prev_letter = prev_var.letters[prev_letter_idx]
                curr_letter = candidate[idx]
                candidate_log_prob += TEMP_BI  #bigram_probs[prev_letter][curr_letter]

            if idx < self.length - 1:  # Next letter bigram
                next_var = self.neighbors[idx + 1]
                next_letter_idx = candidate_indices[idx + 1]
                next_letter = next_var.letters[next_letter_idx]
                curr_letter = candidate[idx]
                candidate_log_prob += TEMP_BI  #bigram_probs[curr_letter][next_letter]

            message[candidate_indices[idx]] = logsumexp([message[candidate_indices[idx]], candidate_log_prob])

        self.messages[variable_node.position] = message - logsumexp(message)
        variable_node.receive_message(self, self.messages[variable_node.position])


class Crossword:
    def __init__(self, grid, variables, factors):
        self.grid = grid
        self.variables = variables
        self.factors = factors

    def run_belief_propagation(self, num_iterations=10):
        for _ in range(num_iterations):
            for factor in self.factors.values():
                for neighbor in factor.neighbors:
                    factor.send_message(neighbor)

            for variable in self.variables.values():
                variable.normalize()

    def get_solution(self):
        solution = {}
        for position, variable in self.variables.items():
            best_letter = variable.letters[np.argmax(variable.log_probs)]
            solution[position] = best_letter
        return solution

variables = {}
factors = {}

# Create VariableNodes for each cell
for row in range(4):
    for col in range(4):
        position = (row, col)
        variables[position] = VariableNode(position)

# Create FactorNodes for each clue
clues = {
    '1A': [(0, 0), (0, 1), (0, 2), (0, 3)],
    '2A': [(1, 0), (1, 1), (1, 2), (1, 3)],
    '3A': [(2, 0), (2, 1), (2, 2), (2, 3)],
    '4A': [(3, 0), (3, 1), (3, 2), (3, 3)],
    '1D': [(0, 0), (1, 0), (2, 0), (3, 0)],
    '2D': [(0, 1), (1, 1), (2, 1), (3, 1)],
    '3D': [(0, 2), (1, 2), (2, 2), (3, 2)],
    '4D': [(0, 3), (1, 3), (2, 3), (3, 3)]
}

for clue, positions in clues.items():
    length = len(positions)
    factors[clue] = FactorNode(clue, length, candidates[clue], confidence_ratings[clue])
    for position in positions:
        factors[clue].add_neighbor(variables[position])
        variables[position].add_neighbor(factors[clue])

# Create the crossword object and run belief propagation
crossword = Crossword(grid, variables, factors)
crossword.run_belief_propagation(num_iterations=10)
solution = crossword.get_solution()

# Print the solution
print("Crossword Solution:")
for row in range(4):
    print(''.join(solution[(row, col)] for col in range(4)))

Crossword Solution:
FOUL
ORAL
USED
RAIN


/var/folders/__/45vv5xv57jdctpcpjyw3ybdm0000gn/T/ipykernel_27807/1521317805.py:65: RuntimeWarning: divide by zero encountered in log
  candidate_log_prob = np.log(self.confidence_ratings[candidate])
